<a href="https://colab.research.google.com/github/ahanna13/MS_NN/blob/main/MS_NN_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is my first trial at a neural net for my dataset from RKI:

### Version 4.1 (20230306) of the MALDI-ToF Mass Spectrometry Database for Identification and Classification of Highly Pathogenic Microorganisms from the Robert Koch-Institute (RKI). It can be found at https://zenodo.org/records/7990990

In [ ]:
# DONT RUN THIS CELL ASH
# THIS CELL WAS ORIGINALLY USED TO READ IN THE DATASET VIA FOR LOOPS
# DO NOT NEED TO RUN AGAIN SINCE I SAVED IT TO A HDF5 IN A LATER CELL

# trying to load the data using os.path and numpy
import os
import numpy as np
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

#from sklearn.preprocessing import MultiLabelBinarizer
#from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# initializing a vector to append to
labeled_vector = []

# initializing the right size to also append to
# 66000 total x axis points for all interpolated files; 9823 total files
data = np.zeros((66000,9823),dtype='float')

# set the base path to directory that holds all the genre folders each with many txt files
#basedir = '/content/drive/MyDrive/ColabNotebooks/RKI_V4_1_ExtractedTXTs/Interpolated_RKI_V4_1/'
basedir = "C:\\Users\\Ashley Hanna\\Interpolated_RKI_V4_1\\"

# capture all the folder names since each of those is the label for the txts inside the folder
# aka since I am trying to predict the genus of the microbe analyzed, the data is organized into folders for each genus
labels = os.listdir(basedir)

# initializing a counter
col_num = 0

for (root,dirs,files) in os.walk(basedir):
    for label in labels:
        if os.path.basename(root) == label:
            for file in files:
                print(os.path.join(root,file))
                #df = pd.read_csv(os.path.join(root,file),header=None,sep=" ",dtype='float')
                df = np.loadtxt(os.path.join(root,file))
                print(os.path.basename(root))
                data[:,col_num] = df[:,1]
                col_num += 1
                print(col_num)
                labeled_vector.append(os.path.basename(root))

print(data)
print(labeled_vector)

In [ ]:
# DONT RUN THIS CELL ASH
# THIS CELL WAS USED TO GRAB ONLY TWO GENRE TO SIMPLIFY THE NERUAL NET
# THIS DROPS THE CLASSES FROM 73 TOTAL TO 2
# 73 WAS TOO MANY TO START

# THIS CELL DOES NOT NEED TO BE RUN AGAIN, SAVED TO H5PY FILES

# convert to df with labels as first row and data as the rest
#pd.DataFrame({data[:,i]:j for i,j in enumerate(labeled_vector)})

#unique_genus_labels = np.unique(labeled_vector)

# turning text labels into integer labels
labeled_vector = np.array(labeled_vector)
labeled_encoder = LabelEncoder()
vec = labeled_encoder.fit_transform(labeled_vector)

# grabbing the data for Brucella and Burkholderia spectra only
# this for loop is just counting how many spectra there are in total
# starting a counter
j = 0
new_data = []
new_labeled_encoder = []
for i in range(len(vec)):
    if vec[i]==10: # reminder 6 is bacillus
        #new_labeled_encoder[j] = 0 # calling bac "0"
        #new_labeled_encoder.append(0)
        #new_data[:,j]=data[:,i]
        j = j + 1
    if vec[i]==11:
        #new_labeled_encoder[j] = 1 # calling bac "0"
        #new_labeled_encoder.append(1)
        #new_data[:,j]=data[:,i]
        j = j + 1

new_labeled_encoder = np.zeros(j)
new_data = np.zeros((data.shape[0],j))
j = 0

# initializing counts for each genus
Brucella = 0
Burkholderia = 0

# this for loop is actually grabbing the data out and storing it
for i in range(len(vec)):
    if vec[i]==10:
        new_labeled_encoder[j] = 0
        new_data[:,j]=data[:,i]
        j = j + 1
        Brucella = Brucella + 1
    if vec[i] == 11:
        new_labeled_encoder[j] = 1
        new_data[:,j] = data[:,i]
        j = j + 1
        Burkholderia = Burkholderia + 1

# print out how many there are of each genus
print(Brucella)
print(Burkholderia)

In [ ]:
# DONT RUN THIS CELL ASH
# THIS CELL IS USED TO INTEGER ENCODE AND THEN ONE HOT ENCODE MY LABELS
# SHUFFLE OF DATA OCCURS HERE

#from sklearn.preprocessing import MultiLabelBinarizer
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

#vec = labeled_encoder.fit_transform(labeled_vector)

# shuffle data
indx_rand = np.random.permutation(j)

# integer encode
vec = new_labeled_encoder[indx_rand]
new_data = new_data[:,indx_rand]

# checking the shape
print(np.shape(vec))

# one-hot-encoding
one_hot_label = to_categorical(vec)

#train_df, val_df = train_test_split(data, test_size=0.2)

#X_train = np.transpose(data[:,0::2]); Y_train = one_hot_label[0::2,:];
#X_test  = np.transpose(data[:,1::2]); Y_test  = one_hot_label[1::2,:];

# x = data of the intensitites
# data is a vector with every col being a spectrum
# y = labels

In [ ]:
# DONT RUN THIS CELL ASH
# THIS CELL WAS WHAT WROTE THE DATASET TO A HDF5 FILE TYPE USING H5PY
import h5py

# 'w' means write
# 'r' means read
# https://docs.h5py.org/en/stable/high/file.html

f = h5py.File('data_NN_class_project_3.hdf5','w')

f["data"] = new_data
f["label"] = one_hot_label

f.close()

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ START HERE!!!!!!!!!!!!!!!!!! ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# RUN THIS ONE ASH
# THIS CELL IS WHAT LOADS IN THE DATASET FROM HDF5 FILES
# RUN EVERY TIME KERNAL IS RELOADED
import h5py
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# to view and open data
# https://myhdf5.hdfgroup.org/

# Initialize arrays of the correct size for my data
data_h5py = np.zeros([66000, 1833]) # intensities
one_hot_label_h5py = np.zeros([1833, 2]) # labels

# read in the data from files
with h5py.File("C:\\Users\\Ashley Hanna\\data_NN_class_project_3.hdf5", "r") as f:
    data_h5py[:] = f["data"] # save the intensities for later use
    one_hot_label_h5py[:] = f["label"] # save the labels for later use


In [ ]:
# RUN THIS ONE ASH
# TRAIN-TEST-VALIDATION SPLIT OCCURS HERE
from sklearn.model_selection import train_test_split

# typical to have the first paramater to be the number of the samples, second is number of features
# create test train split
#x_train, x_test, y_train, y_test = train_test_split(np.transpose(new_data), one_hot_label, test_size=0.33, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(np.transpose(data_h5py), one_hot_label_h5py, test_size=0.33, random_state=42)

# create validation split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

#x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
#x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
#y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))
#y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], 1))
#x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))
#y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], 1))


In [ ]:
# RUN THIS ONE ASH
# In Keras, model.fit requires two numpy arrays to train on - data, labels.
# This works correctly if each row of the data has one corresponding label.

model = Sequential([
    # input shape should be 66,000, 1
    Conv1D(4, 4, activation='relu', input_shape=(66000, 1)),  # 32 might be too big, try 8; may not need 32 since its spectral peaks
    #MaxPooling1D(8),
    #Conv1D(16, 16, activation='relu'),
    #MaxPooling1D(8),
    #Conv1D(8, 16, activation='relu'),
    #MaxPooling1D(8),
    Flatten(),
    #Dense(2048, activation='relu'),
    #Dense(256, activation='relu'),
    #Dense(64, activation='relu'),# Ved said to add another hidden layer
    # y_train.shape[1] should be 72 for the number of genera I have
    Dense(2, activation='softmax')  # Assuming y is one-hot encoded
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
# DONT RUN THIS CELL ASH
# Attempted Keras Tuner
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    model = keras.Sequential()

    # Tune the number of convolutional layers
    for i in range(hp.Int('conv_layers', 1, 3, default=2)):
        model.add(layers.Conv1D(
            # number of filters aka number of patterns
            filters=hp.Int(f'filters_{i}', 8, 64, step=8),
            # kernal size aka filter size
            kernel_size=hp.Int(f'kernel_size_{i}', 3, 10),
            activation='relu',
            # feeding in 1 spectra a time (aka 1 col)
            # col length is 66000
            # only looking at one input value for each m/z (channel)
            input_shape=(66000, 1) if i == 0 else None,
            padding='same'
        ))
        #print(layers.output_shape)
       # model.add(layers.MaxPooling1D(pool_size=1))

    model.add(layers.Flatten())

    # Tune the number of dense layers
    for i in range(hp.Int('dense_layers', 1, 3, default=2)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
            activation='relu'
        ))

    model.add(layers.Dense(73, activation='softmax'))

    # Tune the learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
# DONT RUN THIS CELL ASH
# Create a tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of different configurations to try
    executions_per_trial=1,  # Number of models to train per configuration
    directory='./model_tuning',  # Directory to store the models
    project_name='keras_tuner_demo'
)

# Load your data here
# x_train, y_train, x_val, y_val = ...



# Start the tuning process
tuner.search(
    x_train, y_train,
    epochs=10,
    validation_data=(x_val, y_val)
)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Summary of the best model
best_model.summary()

In [ ]:
# RUN THIS ONE ASH
# TRAIN YOUR MODEL
history = model.fit( #best_model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    #validation_split=0.1,
    epochs=5,
    batch_size=128
)

In [ ]:
# RUN THIS ONE ASH
# EVALUATE HOW IT DID VISUALLY W/ LOSS CURVE
import matplotlib.pyplot as plt
# import xarray as xr

plt.plot(history.history["loss"], c='k', label="training")
plt.plot(history.history["val_loss"], c='r', label="validation")
plt.legend(loc="upper right")
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss/Error')
plt.show()

In [ ]:
# RUN THIS ONE ASH
# EVALUATE HOW IT DID VISUALLY W/ ACCURACY CURVE
plt.plot(history.history["accuracy"], c='k', label="training")
plt.plot(history.history["val_accuracy"], c='r', label="validation")
plt.legend(loc="upper right")
plt.title('Accuracy Curve')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.show()

# SHOULD BE PLOTTING CATEGORICAL CROSSENTROPY NOT ACCURACY
# check in the RNNs slide deck

In [ ]:
# RUN THIS ONE ASH
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=36)
print("test loss, test acc:", results)

In [ ]:
# Confusion matrix for NN
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = model.predict(x_test)

# convert to integer labels
#y_pred_int_labels = np.argmax(y_pred)#, axis=1)
y_test_int_labels = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test_int_labels, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)

disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for NN')
plt.show()

In [ ]:
# ROC curve for NN
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Calculate the probabilities of each class
y_probs = model.predict_proba(x_test)[:, 1]  # probabilities for the positive class

fpr, tpr, thresholds = roc_curve(y_test_int_labels, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for NN')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Feature importance for NN
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature Importances for NN")
plt.bar(range(x_train.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(x_train.shape[1]), indices)
plt.xlim([-1, x_train.shape[1]])
plt.show()

In [ ]:
# Random forest for comparison
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

data_h5py_RF = data_h5py
one_hot_label_h5py_RF = one_hot_label_h5py

# transpose dataset to match up the lengths
x_RF = np.transpose(data_h5py_RF)
# Convert one-hot encoded labels to a single label vector
y_RF = np.argmax(one_hot_label_h5py_RF, axis=1)

# Split the data into training and test sets
x_train_RF, x_test_RF, y_train_RF, y_test_RF = train_test_split(x_RF, y_RF, test_size=0.25, random_state=42)

# Create the random forest classifier
RFmodel = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
RFmodel.fit(x_train_RF, y_train_RF)

# Evaluate the model
accuracy = RFmodel.score(x_test_RF, y_test_RF)
print(f'Accuracy: {accuracy:.2f}')

In [ ]:
# Plot the random forest
# Confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred_RF = RFmodel.predict(x_test_RF)

cm = confusion_matrix(y_test_RF, y_pred_RF)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=RFmodel.classes_)

disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for Random Forest')
plt.show()

In [ ]:
# Plot the random forest
# ROC curve
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Calculate the probabilities of each class
y_probs_RF = RFmodel.predict_proba(x_test_RF)[:, 1]  # probabilities for the positive class

fpr, tpr, thresholds = roc_curve(y_test_RF, y_probs_RF)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Random Forest')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Plot the random forest
# Feature importance
importances = RFmodel.feature_importances_
indices = np.argsort(importances)[::-1]

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature Importances for Random Forest")
plt.bar(range(x_train_RF.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(x_train_RF.shape[1]), indices)
plt.xlim([-1, x_train_RF.shape[1]])
plt.show()


# I need to add https://wandb.ai/site